In [18]:
from transformers import TFAutoModelForMaskedLM
from transformers import AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForMaskedLM.

All the weights of TFDistilBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [69]:
tokenizer.mask_token

'[MASK]'

In [24]:
sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")

In [25]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [26]:
tokenizer.mask_token_id

103

In [27]:
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]


In [28]:
mask_token_index

<tf.Tensor: shape=(), dtype=int64, numpy=23>

In [34]:
inputs.input_ids

<tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[  101, 12120,  2050,  8683,  1181,  3584,  1132,  2964,  1190,
         1103,  3584,  1152, 27180,   119,  7993,  1172,  1939,  1104,
         1103,  1415,  3827,  1156,  1494,   103,  1412,  6302,  2555,
        10988,   119,   102]], dtype=int32)>

In [37]:
tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0,1]

<tf.Tensor: shape=(), dtype=int64, numpy=23>

In [46]:
output = model(inputs)
output

TFMaskedLMOutput(loss=None, logits=<tf.Tensor: shape=(1, 30, 28996), dtype=float32, numpy=
array([[[ -6.673225 ,  -6.645008 ,  -6.7923255, ...,  -5.5929737,
          -5.278313 ,  -5.6558537],
        [ -6.322111 ,  -5.637912 ,  -5.899028 , ...,  -4.6864123,
          -4.1498885,  -5.350705 ],
        [ -5.986271 ,  -6.0991263,  -5.808884 , ...,  -5.229702 ,
          -4.3014517,  -6.597116 ],
        ...,
        [ -7.8892307,  -7.671843 ,  -7.6357255, ...,  -6.9082594,
          -5.585291 ,  -6.245915 ],
        [-14.771035 , -14.271366 , -14.164205 , ..., -11.476956 ,
         -12.169214 , -13.1041155],
        [-14.369466 , -13.983858 , -13.632993 , ..., -11.206621 ,
         -11.67536  , -12.708331 ]]], dtype=float32)>, hidden_states=None, attentions=None)

In [49]:
output.logits.shape

TensorShape([1, 30, 28996])

In [67]:
text = ''
for i in range(len(output.logits.numpy()[0])):
  token = np.argmax(output.logits.numpy()[0][i])
  text += " "+tokenizer.decode([token])
text

' . Di ##st ##ille ##d models are smaller than the models they mimic . Using them instead of the large versions would help reduce our carbon foot ##print . .'

In [30]:
token_logits = model(**inputs).logits

In [31]:
# Get the probabilty index of 23 index word
mask_token_logits = token_logits[0, mask_token_index, :]

In [64]:
mask_token_logits

<tf.Tensor: shape=(28996,), dtype=float32, numpy=
array([-5.550159 , -5.6789675, -5.325627 , ..., -5.480676 , -4.510661 ,
       -4.2441115], dtype=float32)>

In [38]:
# get the top 5 index values
top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))
    print("="*10)

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


In [40]:
top_5_tokens

array([ 4851,  2773,  9711, 18134,  4607], dtype=int32)